In [1]:
from Hamil_search import *
from simulation import *
import numpy as np
import os.path
import pennylane as qml

In [2]:
gx, gz = 1, 3

In [3]:
def load_or_precompute(n_block):
    if not os.path.exists(f"Hpen_{n_block}blocks.npy"):
        n_phys = 4 * n_block
        Hpen_terms = []
        for i in range(n_block):
            Hpen_terms += [(gx, f'X{4*i}*X{4*i+1}'), (gz, f'Z{4*i}*Z{4*i+1}'),
                        (gx, f'X{4*i+2}*X{4*i+3}'), (gz, f'Z{4*i+2}*Z{4*i+3}')]
        Hpen = blocks2Mat(n_phys, Hpen_terms)
        HpenInverse = np.linalg.pinv(Hpen)
        Q0 = HpenInverse @ Hpen
        P0 = np.identity(2 ** n_phys) - Q0
        Uenc = getU(4, n_block)
        Penc = Uenc @ Uenc.conj().T
        np.save(f'Hpen_{n_block}blocks', Hpen)
        np.save(f'HpenInverse_{n_block}blocks', HpenInverse)
        np.save(f'Q0_{n_block}blocks', Q0)
        np.save(f'P0_{n_block}blocks', P0)
        np.save(f'Uenc_{n_block}blocks', Uenc)
        np.save(f'Penc_{n_block}blocks', Penc)
    
    Hpen = np.load(f'Hpen_{n_block}blocks.npy')
    HpenInverse = np.load(f'HpenInverse_{n_block}blocks.npy')
    Q0 = np.load(f'Q0_{n_block}blocks.npy')
    P0 = np.load(f'P0_{n_block}blocks.npy')
    Uenc = np.load(f'Uenc_{n_block}blocks.npy')
    Penc = np.load(f'Penc_{n_block}blocks.npy')
    
    return Hpen, HpenInverse, Q0, P0, Uenc, Penc

def test_leakage(HpenInverse, P0, Penc, A):
    off_diag = (P0 - Penc) @ A @ HpenInverse @ A @ Penc
    if checkSame(off_diag, np.zeros(A.shape)):
        print("no leakage")
    else:
        print("yes leakage")

def get_logical_interaction(HpenInverse, Uenc, Henc):
    Hlogi = - Uenc.conj().T @ Henc @ HpenInverse @ Henc @ Uenc
    return qml.pauli_decompose(Hlogi)


In [43]:
lamb = 64
n_block = 2
n_phys = 4 * n_block
n_logi = 2 * n_block
dim_phys = 2 ** n_phys
dim_logi = 2 ** n_logi

Hpen, HpenInverse, Q0, P0, Uenc, Penc = load_or_precompute(n_block)

A_terms = [(1, 'Z1*X6')]
A = blocks2Mat(n_phys, A_terms)

B_terms = [(1, 'Z3*X6')]
B = blocks2Mat(n_phys, B_terms)

C_terms = [(1, 'X1*Z6')]
C = blocks2Mat(n_phys, C_terms)

D_terms = [(1, 'X3*Z6')]
D = blocks2Mat(n_phys, D_terms)

print(checkSame(P0 @ B @ HpenInverse @ A @ P0 + P0 @ A @ HpenInverse @ B @ P0,
                np.zeros((dim_phys, dim_phys))))
Henc = A+B+C+D
test_leakage(HpenInverse, P0, Penc, Henc)
get_logical_interaction(HpenInverse, Uenc, Henc)


False
yes leakage


(
    0.12499999999999993 * (I(0) @ I(1) @ I(2) @ X(3))
  + -0.3749999999999998 * (I(0) @ I(1) @ Z(2) @ I(3))
  + 0.3749999999999998 * (I(0) @ Z(1) @ Z(2) @ I(3))
  + 0.1249999999999999 * (X(0) @ I(1) @ I(2) @ X(3))
)

In [51]:
lamb = 64
n_block = 2
n_phys = 4 * n_block
n_logi = 2 * n_block
dim_phys = 2 ** n_phys
dim_logi = 2 ** n_logi

Hpen, HpenInverse, Q0, P0, Uenc, Penc = load_or_precompute(n_block)

Henc_terms = [(1, 'Z1*X6'), (1, 'Z3*X6'), (3, 'Z1*X4'), (3, 'Z1*X6')]
Henc = blocks2Mat(n_phys, Henc_terms)

test_leakage(HpenInverse, P0, Penc, Henc)
get_logical_interaction(HpenInverse, Uenc, Henc)


no leakage


(
    -1.4999999999999993 * (I(0) @ I(1) @ Z(2) @ I(3))
  + -1.4999999999999996 * (I(0) @ X(1) @ I(2) @ I(3))
  + 1.4999999999999996 * (I(0) @ X(1) @ X(2) @ I(3))
  + 1.4999999999999993 * (I(0) @ Z(1) @ Z(2) @ I(3))
)

In [4]:
n_block = 2
n_phys = 4 * n_block

Hpen_terms = [(1, 'X0*X1'), (3, 'Z0*Z1'), (1, 'X2*X3'), (3, 'Z2*Z3'),
              (2, 'X4*X5'), (6, 'Z4*Z5'), (2, 'X6*X7'), (6, 'Z6*Z7')]
Hpen = blocks2Mat(n_phys, Hpen_terms)
HpenInverse = np.linalg.pinv(Hpen)
Q0 = HpenInverse @ Hpen
P0 = np.identity(2 ** n_phys) - Q0
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T


Henc_terms = [(1, 'Z1*X6'), (1, 'Z3*X6')]
Henc = blocks2Mat(n_phys, Henc_terms)

test_leakage(HpenInverse, P0, Penc, Henc)
get_logical_interaction(HpenInverse, Uenc, Henc)

no leakage


(
    -0.17142857142857137 * (I(0) @ I(1) @ Z(2) @ I(3))
  + 0.17142857142857137 * (I(0) @ Z(1) @ Z(2) @ I(3))
)

In [5]:
n_block = 3
n_phys = 4 * n_block

Hpen_terms = [(1, 'X0*X1'), (3, 'Z0*Z1'), (1, 'X2*X3'), (3, 'Z2*Z3'), # block 0
              (1, 'X4*X5'), (3, 'Z4*Z5'), (1, 'X6*X7'), (3, 'Z6*Z7'), # block 1
              (1, 'X8*X9'), (3, 'Z8*Z9'), (1, 'X10*X11'), (3, 'Z10*Z11')] # block 2
Hpen = blocks2Mat(n_phys, Hpen_terms)
HpenInverse = np.linalg.pinv(Hpen)
Q0 = HpenInverse @ Hpen
P0 = np.identity(2 ** n_phys) - Q0
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T


Henc_terms = [(1, 'Z1*X6'), (1, 'Z3*X6'),  # cross-block gadget between block 0 & block 1
              (1, 'X9*Z6'), (1, 'X11*Z6')] # cross-block gadget between block 1 & block 2
Henc = blocks2Mat(n_phys, Henc_terms)

test_leakage(HpenInverse, P0, Penc, Henc)

yes leakage


In [6]:
n_block = 3
n_phys = 4 * n_block

Hpen_terms = [(1, 'X0*X1'), (3, 'Z0*Z1'), (1, 'X2*X3'), (3, 'Z2*Z3'), # block 0
              (1, 'X4*X5'), (3, 'Z4*Z5'), (1, 'X6*X7'), (3, 'Z6*Z7'), # block 1
              (2, 'X8*X9'), (6, 'Z8*Z9'), (2, 'X10*X11'), (6, 'Z10*Z11')] # block 2
Hpen = blocks2Mat(n_phys, Hpen_terms)
HpenInverse = np.linalg.pinv(Hpen)
Q0 = HpenInverse @ Hpen
P0 = np.identity(2 ** n_phys) - Q0
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T


Henc_terms = [(1, 'Z1*X6'), (1, 'Z3*X6'),  # cross-block gadget between block 0 & block 1: should yield logical ZZ
              (1, 'X9*Z6'), (1, 'X11*Z6')] # cross-block gadget between block 1 & block 2: should yield logical XX
Henc = blocks2Mat(n_phys, Henc_terms)

test_leakage(HpenInverse, P0, Penc, Henc)
get_logical_interaction(HpenInverse, Uenc, Henc)

no leakage


(
    0.028571428571428623 * (I(0) @ I(1) @ I(2) @ X(3) @ I(4) @ I(5))
  + 0.02857142857142862 * (I(0) @ I(1) @ I(2) @ X(3) @ X(4) @ I(5))
  + -0.3749999999999999 * (I(0) @ I(1) @ Z(2) @ I(3) @ I(4) @ I(5))
  + 0.3749999999999999 * (I(0) @ Z(1) @ Z(2) @ I(3) @ I(4) @ I(5))
)